# Comprehensive SyftBox Permissions Tutorial
## Part 4: Terminal Nodes and File Limits

Welcome to Part 4! This is where you'll learn the most advanced SyftBox features: terminal nodes that can block inheritance completely, and file limits that control what types and sizes of files are allowed.

### What You'll Learn
- Terminal nodes and inheritance blocking
- File size limits and enforcement
- Directory and symlink restrictions
- Advanced access control patterns
- Real-world security scenarios

### Prerequisites
- Completed Parts 1-3
- Strong understanding of inheritance and nearest-node algorithm

In [ ]:
import syft_perm as sp
from pathlib import Path
import tempfile
import shutil
import yaml

# Setup workspace
tutorial_dir = Path(tempfile.mkdtemp(prefix="terminal_tutorial_"))
print(f"Tutorial workspace: {tutorial_dir}")

def cleanup_and_reset():
    global tutorial_dir
    if tutorial_dir.exists():
        shutil.rmtree(tutorial_dir)
    tutorial_dir = Path(tempfile.mkdtemp(prefix="terminal_tutorial_"))
    print(f"Fresh workspace: {tutorial_dir}")
    return tutorial_dir

def show_yaml(path, title=None):
    yaml_file = path / "syft.pub.yaml"
    if yaml_file.exists():
        display_title = title or f"syft.pub.yaml in {path.name}"
        print(f"\n=== {display_title} ===")
        print(yaml_file.read_text())
        print("=" * 50)

def test_access_matrix(files, users, permission="read"):
    """Test access for multiple files and users in a matrix"""
    print(f"\n=== Access Matrix ({permission.upper()}) ===")
    
    # Header
    header = f"{'File':<25}"
    for user in users:
        header += f"{user.split('@')[0]:<12}"
    print(header)
    print("-" * len(header))
    
    # Test each file
    for rel_path, full_path in files.items():
        syft_file = sp.open(full_path)
        row = f"{rel_path:<25}"
        
        for user in users:
            has_access = getattr(syft_file, f"has_{permission}_access")(user)
            symbol = "✓" if has_access else "✗"
            row += f"{symbol:<12}"
        
        print(row)

## Chapter 1: Understanding Terminal Nodes

A **terminal node** is a directory that completely blocks inheritance from parent directories. When you set `terminal: true` in a `syft.pub.yaml`, that directory becomes a "firewall" that stops all parent permissions.

In [ ]:
# Create a scenario to demonstrate terminal nodes
# Structure:
# tutorial_dir/ (public access)
#   ├── public.txt
#   ├── restricted/ (terminal node)
#   │   ├── secret.txt
#   │   └── classified.txt
#   └── normal/
#       └── inherited.txt

files = {
    "public.txt": tutorial_dir / "public.txt",
    "restricted/secret.txt": tutorial_dir / "restricted" / "secret.txt",
    "restricted/classified.txt": tutorial_dir / "restricted" / "classified.txt",
    "normal/inherited.txt": tutorial_dir / "normal" / "inherited.txt"
}

for rel_path, full_path in files.items():
    full_path.parent.mkdir(parents=True, exist_ok=True)
    full_path.write_text(f"Content of {rel_path}")

print("Created directory structure:")
print(tutorial_dir.name + "/")
print("├── public.txt")
print("├── restricted/ (will be terminal)")
print("│   ├── secret.txt")
print("│   └── classified.txt")
print("└── normal/")
print("    └── inherited.txt")

In [ ]:
# Set up permissions: Root gives public access
root_yaml = tutorial_dir / "syft.pub.yaml"
root_yaml.write_text("""rules:
- pattern: "**"
  access:
    read:
    - "*"  # Everyone can read everything
""")

# Make restricted/ a terminal node with specific access
restricted_yaml = tutorial_dir / "restricted" / "syft.pub.yaml"
restricted_yaml.write_text("""terminal: true  # This blocks inheritance!
rules:
- pattern: "secret.txt"
  access:
    read:
    - admin@example.com
- pattern: "classified.txt"
  access:
    read:
    - topsecret@example.com
""")

show_yaml(tutorial_dir, "Root: Public access to everything")
show_yaml(tutorial_dir / "restricted", "Restricted: Terminal with specific rules")

print("\n🔥 Key Point: 'terminal: true' completely blocks the public access from root!")

In [ ]:
# Test the terminal node behavior
users = ["public_user@example.com", "admin@example.com", "topsecret@example.com"]

test_access_matrix(files, users, "read")

print("\n📊 Analysis:")
print("✓ public.txt and normal/inherited.txt: Public access works (inherits from root)")
print("✗ restricted/secret.txt: Only admin has access (terminal blocks public)")
print("✗ restricted/classified.txt: Only topsecret has access (terminal blocks public)")
print("\nThe terminal node completely overrides parent permissions!")

In [ ]:
# Show detailed reasoning for terminal blocking
print("\n=== Detailed Terminal Node Analysis ===")

secret_file = sp.open(tutorial_dir / "restricted" / "secret.txt")
normal_file = sp.open(tutorial_dir / "normal" / "inherited.txt")

print("\n1. Normal file (inherits from root):")
has_read, reasons = normal_file._check_permission_with_reasons("public_user@example.com", "read")
print(f"   Access: {has_read}")
for reason in reasons:
    print(f"   • {reason}")

print("\n2. File in terminal node:")
has_read, reasons = secret_file._check_permission_with_reasons("public_user@example.com", "read")
print(f"   Access: {has_read}")
for reason in reasons:
    print(f"   • {reason}")

print("\n💡 Notice: Terminal node blocks inheritance and creates its own permission island")

## Chapter 2: File Size Limits

SyftBox can enforce file size limits to prevent abuse. This is crucial for shared file systems where storage and bandwidth matter.

In [ ]:
# Create files of different sizes to test limits
cleanup_and_reset()

uploads_dir = tutorial_dir / "uploads"
uploads_dir.mkdir()

# Create files of different sizes
size_test_files = {
    "small.txt": (uploads_dir / "small.txt", "x" * 100),      # 100 bytes
    "medium.txt": (uploads_dir / "medium.txt", "x" * 1000),   # 1KB
    "large.txt": (uploads_dir / "large.txt", "x" * 10000),    # 10KB
    "huge.txt": (uploads_dir / "huge.txt", "x" * 100000),     # 100KB
}

for name, (path, content) in size_test_files.items():
    path.write_text(content)
    size_kb = len(content) / 1024
    print(f"Created {name}: {size_kb:.1f} KB")

print(f"\nAll files in: {uploads_dir.relative_to(tutorial_dir)}/")

In [ ]:
# Set up file size limits
uploads_yaml = uploads_dir / "syft.pub.yaml"
uploads_yaml.write_text("""rules:
- pattern: "*.txt"
  access:
    create:
    - users@example.com
    read:
    - users@example.com
  limits:
    max_file_size: 5120  # 5KB limit
    allow_dirs: true
    allow_symlinks: false
""")

show_yaml(uploads_dir, "Uploads: 5KB size limit")

print("\n=== Testing File Size Limits ===")
print("Rule: Max 5KB files allowed")
print("Expected: small.txt and medium.txt should work, large.txt and huge.txt blocked")

In [ ]:
# Test file size enforcement
user = "users@example.com"

for name, (path, content) in size_test_files.items():
    syft_file = sp.open(path)
    has_access = syft_file.has_read_access(user)
    
    size_kb = len(content) / 1024
    status = "✓" if has_access else "✗"
    expected = "✓" if size_kb <= 5 else "✗"
    
    print(f"{status} {name} ({size_kb:.1f} KB): Access = {has_access} (Expected: {expected == '✓'})")
    
    # Show reasoning for blocked files
    if not has_access:
        has_read, reasons = syft_file._check_permission_with_reasons(user, "read")
        print(f"    Reason: {reasons[0] if reasons else 'Unknown'}")

print("\n📏 Size limits are enforced during permission resolution!")
print("Files exceeding the limit are blocked completely.")

## Chapter 3: File Limits API

You can programmatically set and get file limits using the syft-perm API.

In [ ]:
# Create a file to work with limits API
cleanup_and_reset()

api_file = tutorial_dir / "api_test.txt"
api_file.write_text("Testing limits API")

syft_api = sp.open(api_file)

print("=== File Limits API Demo ===")
print(f"Working with: {api_file.name}")

# Check initial limits (should be none)
initial_limits = syft_api.get_file_limits()
print(f"\nInitial limits: {initial_limits}")

In [ ]:
# Set some permissions first
syft_api.grant_create_access("users@example.com", force=True)

# Set file limits using the API
syft_api.set_file_limits(
    max_size=2048,        # 2KB limit
    allow_dirs=False,     # No directories
    allow_symlinks=False  # No symlinks
)

print("Set limits: 2KB max, no dirs, no symlinks")

# Check the limits
current_limits = syft_api.get_file_limits()
print(f"\nCurrent limits: {current_limits}")

# Show in HTML format (for notebooks)
print("\nHTML representation:")
print(syft_api._repr_html_())

In [ ]:
# Demonstrate partial updates (preserving existing values)
print("\n=== Partial Limit Updates ===")

# Update only the size limit, keep other settings
syft_api.set_file_limits(max_size=4096)  # Double the size limit

updated_limits = syft_api.get_file_limits()
print(f"After size update: {updated_limits}")
print("✓ Size changed to 4096, other settings preserved")

# Update only directory setting
syft_api.set_file_limits(allow_dirs=True)  # Allow directories now

final_limits = syft_api.get_file_limits()
print(f"\nAfter directory update: {final_limits}")
print("✓ Directories now allowed, size and symlink settings preserved")

## Chapter 4: Advanced Terminal Node Scenarios

Terminal nodes can create complex permission boundaries. Let's explore real-world scenarios.

In [ ]:
# Create a complex organizational structure
cleanup_and_reset()

# Structure:
# company/ (general access)
#   ├── public/
#   │   └── announcements.txt
#   ├── departments/
#   │   ├── hr/ (terminal - HR only)
#   │   │   ├── salaries.csv
#   │   │   └── reviews.txt
#   │   ├── finance/ (terminal - Finance only) 
#   │   │   ├── budget.xlsx
#   │   │   └── projections.txt
#   │   └── it/
#   │       └── servers.txt
#   └── projects/
#       ├── alpha/
#       │   └── code.py
#       └── beta/ (terminal - Beta team only)
#           └── secret_feature.py

org_structure = {
    "public/announcements.txt": "Company announcements",
    "departments/hr/salaries.csv": "Employee salaries",
    "departments/hr/reviews.txt": "Performance reviews", 
    "departments/finance/budget.xlsx": "Annual budget",
    "departments/finance/projections.txt": "Financial projections",
    "departments/it/servers.txt": "Server documentation",
    "projects/alpha/code.py": "Alpha project code",
    "projects/beta/secret_feature.py": "Beta secret feature"
}

for rel_path, content in org_structure.items():
    full_path = tutorial_dir / rel_path
    full_path.parent.mkdir(parents=True, exist_ok=True)
    full_path.write_text(content)

print("Created organizational structure with", len(org_structure), "files")
print("\nDirectory layout:")
print("company/")
print("├── public/ (everyone)")
print("├── departments/")
print("│   ├── hr/ (terminal - HR only)")
print("│   ├── finance/ (terminal - Finance only)")
print("│   └── it/ (inherits company access)")
print("└── projects/")
print("    ├── alpha/ (inherits company access)")
print("    └── beta/ (terminal - Beta team only)")

In [ ]:
# Set up the permission hierarchy

# Company root: All employees have basic access
company_yaml = tutorial_dir / "syft.pub.yaml"
company_yaml.write_text("""rules:
- pattern: "public/**"
  access:
    read:
    - "*"  # Public announcements for everyone
- pattern: "**"
  access:
    read:
    - employees@company.com  # Default employee access
""")

# HR department: Terminal with strict access
hr_yaml = tutorial_dir / "departments" / "hr" / "syft.pub.yaml"
hr_yaml.write_text("""terminal: true
rules:
- pattern: "**"
  access:
    admin:
    - hr@company.com
    read:
    - hr_staff@company.com
""")

# Finance department: Terminal with strict access
finance_yaml = tutorial_dir / "departments" / "finance" / "syft.pub.yaml"
finance_yaml.write_text("""terminal: true
rules:
- pattern: "**"
  access:
    admin:
    - cfo@company.com
    write:
    - finance_team@company.com
""")

# Beta project: Terminal for secret development
beta_yaml = tutorial_dir / "projects" / "beta" / "syft.pub.yaml"
beta_yaml.write_text("""terminal: true
rules:
- pattern: "*.py"
  access:
    write:
    - beta_team@company.com
    read:
    - beta_lead@company.com
""")

print("Set up organizational permissions:")
show_yaml(tutorial_dir, "Company: Employee access")
show_yaml(tutorial_dir / "departments" / "hr", "HR: Terminal - HR only")
show_yaml(tutorial_dir / "departments" / "finance", "Finance: Terminal - Finance only")
show_yaml(tutorial_dir / "projects" / "beta", "Beta: Terminal - Beta team only")

In [ ]:
# Test the organizational access control
org_users = [
    "public@internet.com",
    "employees@company.com", 
    "hr@company.com",
    "finance_team@company.com",
    "beta_team@company.com"
]

# Create file objects for testing
org_files = {}
for rel_path in org_structure.keys():
    org_files[rel_path] = tutorial_dir / rel_path

test_access_matrix(org_files, org_users, "read")

print("\n🏢 Organizational Security Analysis:")
print("✓ Public announcements: Everyone can read")
print("✓ IT servers: Employees can read (inherits from company)")
print("✓ Alpha project: Employees can read (inherits from company)")
print("✗ HR files: Only HR team (terminal blocks employee access)")
print("✗ Finance files: Only finance team (terminal blocks employee access)")
print("✗ Beta project: Only beta team (terminal blocks employee access)")
print("\nTerminal nodes create secure departmental boundaries!")

## Chapter 5: Combining Limits and Terminal Nodes

The most powerful scenarios combine terminal nodes with file limits for comprehensive access control.

In [ ]:
# Create a file upload system with different user tiers
cleanup_and_reset()

# Structure:
# uploads/
#   ├── public/ (small files, everyone)
#   ├── premium/ (terminal - larger files, premium users)
#   └── enterprise/ (terminal - unlimited, enterprise users)

upload_dirs = ["public", "premium", "enterprise"]
for dir_name in upload_dirs:
    (tutorial_dir / "uploads" / dir_name).mkdir(parents=True)
    
    # Create sample files of different sizes
    small_file = tutorial_dir / "uploads" / dir_name / "small.txt"
    large_file = tutorial_dir / "uploads" / dir_name / "large.txt"
    huge_file = tutorial_dir / "uploads" / dir_name / "huge.txt"
    
    small_file.write_text("x" * 1000)    # 1KB
    large_file.write_text("x" * 50000)   # 50KB
    huge_file.write_text("x" * 500000)   # 500KB

print("Created tiered upload system:")
print("uploads/")
print("├── public/ (1KB limit)")
print("├── premium/ (100KB limit)")
print("└── enterprise/ (no limit)")
print("\nEach directory has: small.txt (1KB), large.txt (50KB), huge.txt (500KB)")

In [ ]:
# Set up tiered access with limits

# Public: 1KB limit, everyone
public_yaml = tutorial_dir / "uploads" / "public" / "syft.pub.yaml"
public_yaml.write_text("""rules:
- pattern: "*.txt"
  access:
    create:
    - "*"
    read:
    - "*"
  limits:
    max_file_size: 1024  # 1KB limit
    allow_dirs: false
    allow_symlinks: false
""")

# Premium: Terminal with 100KB limit
premium_yaml = tutorial_dir / "uploads" / "premium" / "syft.pub.yaml"
premium_yaml.write_text("""terminal: true
rules:
- pattern: "*.txt"
  access:
    create:
    - premium@example.com
    read:
    - premium@example.com
  limits:
    max_file_size: 102400  # 100KB limit
    allow_dirs: false
    allow_symlinks: false
""")

# Enterprise: Terminal with no limits
enterprise_yaml = tutorial_dir / "uploads" / "enterprise" / "syft.pub.yaml"
enterprise_yaml.write_text("""terminal: true
rules:
- pattern: "*.txt"
  access:
    admin:
    - enterprise@example.com
  # No limits section = no restrictions
""")

print("Set up tiered upload system:")
show_yaml(tutorial_dir / "uploads" / "public", "Public: 1KB limit, everyone")
show_yaml(tutorial_dir / "uploads" / "premium", "Premium: 100KB limit, premium users")
show_yaml(tutorial_dir / "uploads" / "enterprise", "Enterprise: No limits, enterprise users")

In [ ]:
# Test the tiered upload system
print("\n=== Tiered Upload System Test ===")

tiers = ["public", "premium", "enterprise"]
files = ["small.txt", "large.txt", "huge.txt"]
users = {
    "free@example.com": "Free user",
    "premium@example.com": "Premium user", 
    "enterprise@example.com": "Enterprise user"
}

for tier in tiers:
    print(f"\n--- {tier.upper()} TIER ---")
    
    for filename in files:
        file_path = tutorial_dir / "uploads" / tier / filename
        syft_file = sp.open(file_path)
        
        file_size = file_path.stat().st_size
        size_kb = file_size / 1024
        
        print(f"\n{filename} ({size_kb:.0f}KB):")
        
        for user_email, user_desc in users.items():
            has_access = syft_file.has_read_access(user_email)
            status = "✓" if has_access else "✗"
            print(f"  {status} {user_desc}: {has_access}")
            
            # Show why access was denied
            if not has_access:
                has_read, reasons = syft_file._check_permission_with_reasons(user_email, "read")
                if reasons and "limit" in reasons[0].lower():
                    print(f"      → Size limit exceeded")
                elif reasons and "no permission" in reasons[0].lower():
                    print(f"      → Not authorized for this tier")

## Chapter 6: Real-World Security Patterns

Let's explore common security patterns using terminal nodes and limits.

In [ ]:
print("=== Common Security Patterns with Terminal Nodes and Limits ===")
print()

patterns = [
    {
        "name": "Secure Sandbox",
        "use_case": "Isolate untrusted user uploads",
        "config": {
            "terminal": True,
            "limits": {"max_file_size": 1024, "allow_dirs": False, "allow_symlinks": False},
            "access": "Limited users only"
        },
        "why": "Prevents privilege escalation and resource abuse"
    },
    {
        "name": "Department Firewall", 
        "use_case": "Strict departmental boundaries",
        "config": {
            "terminal": True,
            "limits": None,
            "access": "Department members only"
        },
        "why": "Blocks inheritance, ensures data confidentiality"
    },
    {
        "name": "Graduated Access",
        "use_case": "Different limits for different user tiers", 
        "config": {
            "terminal": True,
            "limits": "Varies by tier",
            "access": "Tier-specific users"
        },
        "why": "Monetization and resource management"
    },
    {
        "name": "Admin Override",
        "use_case": "Admin access despite restrictions",
        "config": {
            "terminal": False,
            "limits": None,
            "access": "Admin pattern at higher level"
        },
        "why": "Ensures admin can always access for maintenance"
    }
]

for i, pattern in enumerate(patterns, 1):
    print(f"{i}. **{pattern['name']}**")
    print(f"   Use case: {pattern['use_case']}")
    print(f"   Terminal: {pattern['config']['terminal']}")
    print(f"   Limits: {pattern['config']['limits']}")
    print(f"   Access: {pattern['config']['access']}")
    print(f"   Why: {pattern['why']}")
    print()

In [ ]:
# Demonstrate the "Admin Override" pattern
cleanup_and_reset()

# Create a restricted area with admin override
restricted_file = tutorial_dir / "restricted" / "confidential.txt"
restricted_file.parent.mkdir(parents=True)
restricted_file.write_text("Confidential information")

# Root: Admin can access everything
root_yaml = tutorial_dir / "syft.pub.yaml"
root_yaml.write_text("""rules:
- pattern: "**"
  access:
    admin:
    - admin@company.com  # Admin override
""")

# Restricted: Terminal that blocks normal users
restricted_yaml = tutorial_dir / "restricted" / "syft.pub.yaml"
restricted_yaml.write_text("""terminal: true
rules:
- pattern: "*.txt"
  access:
    read:
    - authorized@company.com  # Only specific user
""")

show_yaml(tutorial_dir, "Root: Admin override")
show_yaml(tutorial_dir / "restricted", "Restricted: Terminal blocking")

print("\n=== Admin Override Test ===")
syft_restricted = sp.open(restricted_file)

test_users = [
    ("admin@company.com", "Admin", True),
    ("authorized@company.com", "Authorized user", True),
    ("employee@company.com", "Regular employee", False)
]

for user, desc, expected in test_users:
    has_access = syft_restricted.has_admin_access(user) or syft_restricted.has_read_access(user)
    status = "✓" if has_access == expected else "✗ WRONG"
    print(f"{status} {desc}: {has_access}")
    
    if has_access:
        has_read, reasons = syft_restricted._check_permission_with_reasons(user, "read")
        print(f"    Reason: {reasons[0] if reasons else 'Unknown'}")

print("\n⚠️  Note: Terminal nodes still block admin inheritance!")
print("For true admin override, admin rules must be within the terminal node.")

## Summary: Part 4 Key Concepts

Fantastic! You've mastered the most advanced SyftBox features:

### ✅ **Terminal Nodes Mastered**
1. **Complete Inheritance Blocking**: `terminal: true` stops all parent permissions
2. **Permission Islands**: Terminal nodes create isolated permission zones
3. **Security Boundaries**: Perfect for departmental or tier-based access
4. **Override Protection**: Even admins can't inherit through terminals

### ✅ **File Limits Mastered**
1. **Size Enforcement**: `max_file_size` blocks oversized files
2. **Type Controls**: `allow_dirs` and `allow_symlinks` for security
3. **API Integration**: Programmatic limit management with get/set methods
4. **Permission Integration**: Limits work with all permission levels

### ✅ **Advanced Patterns**
1. **Tiered Access**: Different limits for different user classes
2. **Organizational Security**: Department firewalls with terminals
3. **Upload Controls**: Size and type restrictions for file uploads
4. **Secure Sandboxes**: Isolated areas with strict controls

### ✅ **Security Best Practices**
1. **Defense in Depth**: Combine terminals, limits, and patterns
2. **Principle of Least Privilege**: Grant minimum necessary access
3. **Resource Protection**: Use size limits to prevent abuse
4. **Clear Boundaries**: Use terminals for organizational separation

### 🚀 **Next Up: Part 5 - Complex Scenarios and Debugging**
In the final tutorial, you'll learn:
- Real-world complex scenarios
- Advanced debugging techniques
- Performance considerations
- Best practices and common pitfalls

You're now a SyftBox permissions expert! 🎉

In [ ]:
# Cleanup
shutil.rmtree(tutorial_dir)
print("Tutorial 4 complete! 🎉")
print("Ready for Part 5: Complex Scenarios and Mastery")